In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_rows', 40)

## Read csv files from db directory

In [3]:
# file and DB URL
db_url = "C:/Users/hyo05/Documents/Portfolio/Apple transaction/csv/"
file_url = db_url + "Apple Card Transactions - April 2021.csv"
print(file_url)

C:/Users/hyo05/Documents/Portfolio/Apple transaction/csv/Apple Card Transactions - April 2021.csv


# 1.2 Transaction datre
Step 2. 

# Step 1. Data cleaning

## 1.1 Reading a csv file and create data frame

### Rematch falsely matched pair of Merchant and Category.  

In [4]:
# Define mecrchant and category dictionary for pair of merchant and category that does not match up with each other in the original data set.
apple_merchant_dict = {'Apple Services' : 'Entertainment', 'Extra Space Storage' : 'Utilities', 'Geico' : 'Automobile', 'City of LA Dot Pvb Pbp' : 'Automobile', 'Soundcloud' : 'Subscriptions', 'T-Mobile' : 'Utilities'}

In [11]:
# Search for merchants in the dictionary and edit category if the merchant exists. 
def match_category(df, dict):
    for index, row in df.iterrows():
        if row['Merchant'] in dict.keys():
            df.at[index, 'Category'] = dict.get(row['Merchant'])
        elif row['Merchant'] == "Ach Deposit Internet Transfer From Account Ending In 3437":
            df.at[index, 'Merchant'] = "Card payment"
    return df

### Divide 'Transaction Date' column into 3 columns: Month, Day, Year

In [12]:
# Cleaning date into Month, Day, Year for future operations
def clean_date(df):

    month = np.array([]).astype(int)
    day = np.array([]).astype(int)
    year = np.array([]).astype(int)

    for index, row in df.iterrows():
        date = row['Transaction Date'].split('/')
        month = np.append(month, date[0])
        day = np.append(day, date[1])
        year = np.append(year, date[2])

    # Insert Month, Day, Year column and drop 'Transaction Date' column.
    df.insert(0, 'Month', month)
    df.insert(1, 'Day', day)
    df.insert(2, 'Year', year)
    df = df.drop(columns=['Transaction Date'])

    return df

In [27]:
def read_csv(f_url):
    # Reading csv file for selected columns and create and sort dataframe by the Transaction date
    f_df = pd.read_csv(f_url)[['Transaction Date', 'Merchant', 'Category', 'Type', 'Amount (USD)']]
    f_df = f_df.sort_values(by=['Transaction Date'])

    # Cleaning Merchant and Category
    f_df = match_category(f_df, apple_merchant_dict)

    # Clean 'Transaction Date' column. 
    f_df = clean_date(f_df)

    return f_df

april_df = read_csv(file_url)
print(april_df)

   Month Day  Year        Merchant       Category      Type  Amount (USD)
40    03  31  2021        T-Mobile      Utilities  Purchase         26.68
39    04  01  2021           Geico     Automobile  Purchase        264.66
38    04  02  2021      Soundcloud  Subscriptions  Purchase          4.99
36    04  03  2021  Apple Services  Entertainment  Purchase          3.99
37    04  04  2021    Card payment        Payment   Payment       -343.30
..   ...  ..   ...             ...            ...       ...           ...
4     04  29  2021     Papa John's    Restaurants  Purchase         21.84
3     04  29  2021    Seoul Hotdog    Restaurants  Purchase         10.90
2     04  29  2021  Apple Services  Entertainment  Purchase         19.99
1     04  29  2021  Apple Services  Entertainment  Purchase          4.99
0     04  30  2021  Apple Services  Entertainment  Purchase          2.99

[41 rows x 7 columns]


## 1.2 Read all csv files from DB.

### Read all files from DB.

In [31]:
for i in os.listdir(os.getcwd()+'\csv'):
    print(i)

Apple Card Transactions - April 2021.csv


In [ ]:
def read_db():
    return ...

# 2. Analysis
## 2.1 By type
### purchase and payment
## 2.2 Merchant and Disctionary
### Unique merchant
#### Update merchant dictionary
### Monthly spending on each merchant and category 
## 2.3 Monthly report 

In [25]:
purchase = april_df[april_df['Type'] == "Purchase"]
print(purchase)
payment = april_df[april_df['Type'] == "Payment"]
print(payment)

   Month Day  Year                Merchant       Category      Type  \
40    03  31  2021                T-Mobile      Utilities  Purchase   
39    04  01  2021                   Geico     Automobile  Purchase   
38    04  02  2021              Soundcloud  Subscriptions  Purchase   
36    04  03  2021          Apple Services  Entertainment  Purchase   
35    04  08  2021          Apple Services  Entertainment  Purchase   
34    04  08  2021          Apple Services  Entertainment  Purchase   
33    04  13  2021          Apple Services  Entertainment  Purchase   
31    04  18  2021     Extra Space Storage      Utilities  Purchase   
29    04  19  2021                 Safeway        Grocery  Purchase   
28    04  20  2021          Apple Services  Entertainment  Purchase   
30    04  20  2021          Apple Services  Entertainment  Purchase   
26    04  21  2021          Apple Services  Entertainment  Purchase   
25    04  21  2021          Apple Services  Entertainment  Purchase   
24    